In [1]:
import pandas as pd
from itertools import cycle
import gpflow
import numpy as np
from gpflow.utilities import print_summary, positive
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import statsmodels.api as sm
import dill as pickle

sns.set(style="white")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_summary_fmt("notebook")

<ipython-input-1-06da3225ef62>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
final_data = pd.read_pickle("tmp/final_data.pkl")
u0 = pd.read_pickle("tmp/u0.pkl")
#linear_filters_states = pickle.load(open('tmp/linear_filters_states.pkl', 'rb'))
descriptive_stats = pd.read_pickle("tmp/residual_descriptive_stats.pkl")

In [3]:
ORDER = len(list(linear_filters_states.values())[0][0])
N = len(linear_filters_states)

## Rhos

In [5]:
results_states = pickle.load(open('tmp/trained_models_states.pkl', 'rb'))


In [7]:
results_states.keys()

dict_keys(['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas'])

In [ ]:

state_residual = results_states["Alabama"][1]
ORDER = int(descriptive_stats.loc[descriptive_stats.state=="Alabama", "optimal lag"])
rho_, sigma_ = sm.regression.yule_walker(state_residual, order=ORDER, method="mle")
len

In [9]:
from functools import partial

In [12]:
linear_filters_states = {}

ORDER = 5

for state in results_states.keys():
    u0_ = u0[u0['Updated'].isin(final_data.loc[state, 'Updated'])]
    state_residual = results_states[state][1]
    t = u0_["Updated"].index.astype('float').values.reshape(-1, 1)

    def _yule_walker(u=None, coeff_=None, sigma_=None):
        return np.convolve(coeff_, u, mode="same")

    rho_, sigma_ = sm.regression.yule_walker(state_residual, order=ORDER, method="mle")
    rho_ = np.array(rho_)[::-1]
    linear_filters_states[state] = rho_, partial(
        _yule_walker, u=u0_["Residual"].values, coeff_=tuple(rho_), sigma_=sigma_
    )
        

#pickle.dump(linear_filters_states, open('tmp/linear_filters_states.pkl', 'wb'))

In [16]:
for i, (hX, epsX) in enumerate(linear_filters_states.values()):
    print(i, hX)

0 [-0.14643903  0.06704519 -0.16920594  0.12843791  0.56670992]
1 [-0.02558185 -0.22483327  0.28722446 -0.23792546  0.77321569]
2 [-0.06048746  0.07802322 -0.23534824  0.053782    0.41075618]
3 [-0.18796695 -0.00044972  0.04378605  0.10109961  0.14424703]
4 [-0.04629181 -0.03574982 -0.0716829   0.00164463  0.44129902]
5 [-0.27697628  0.18866375  0.07644872 -0.32963489  0.7205469 ]
6 [-0.15102076 -0.13358458 -0.04548212 -0.04226082  0.36235823]
7 [-0.18118596  0.08080095 -0.18567735  0.10321467  0.27770449]
8 [-0.04920707 -0.09264461 -0.0096279   0.19915444  0.6102216 ]
9 [-0.15144291 -0.11152191  0.02694584 -0.13236132  0.37332853]
10 [ 0.01996153 -0.00777803 -0.18114968  0.10314218  0.38960693]
11 [-0.00300286 -0.13448326  0.03933124 -0.18696637  0.51918862]
12 [-0.26272272  0.13068273 -0.01824872 -0.27199511  0.62690492]
13 [-0.06868136  0.01322886  0.00383376 -0.24301964  0.51661272]
14 [-0.19123557  0.05341087 -0.1232227   0.21293816  0.21019968]
15 [ 0.0355288  -0.01787333 -0.0574

In [29]:
-0.06566442*0.27963577

-0.0183621206483034

In [26]:
np.convolve(hX, hX[:-2], mode="same")

array([ 0.01283064,  0.02431542,  0.01277837, -0.01939767, -0.04307371])

In [21]:
np.convolve(hX, hX, mode="same")

array([ 0.02431542,  0.00358065, -0.05144459, -0.08614742, -0.04328058])

In [3]:
times = None
for s in final_data.index.unique():
    s_times = set(final_data.loc[s, 'Updated'].unique())
    if times is None:
        times = s_times
    else:
        times = times.intersection(s_times)

In [5]:
u0_ = u0[u0['Updated'].isin(times)]['Residual'].values
T = u0_.shape[0]
t = np.arange(1, T + 1).reshape(-1, 1)

T2 = T - ORDER + 1

rhos = np.zeros((T2*N, T2*N))
for i, (hX, epsX) in enumerate(linear_filters_states.values()):
    for j, (hY, epsY) in enumerate(linear_filters_states.values()):
        convol = np.convolve(hX, hY[::-1], mode="same")
        for t in range(T2):
            data = convol[max(len(convol)//2 - t, 0): min(len(convol), T2 - t + len(convol)//2)]
            jstart = j*T2+max(t - len(convol)//2, 0)
            jend = jstart + len(data)
            rhos[i*T2+t, jstart:jend] = data

In [6]:
# validation
for x in range(N):
    for y in range(N):
        for t in range(T2):
            for s in range(T2):
                assert rhos[x*T2+t, y*T2+s] == rhos[y*T2+s, x*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{XY}}_{{t,s}} = \rho^{{YX}}_{{s,t}}"
                assert rhos[y*T2+t, x*T2+s] == rhos[x*T2+s, y*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{YX}}_{{t,s}} = \rho^{{XY}}_{{s,t}}"

In [7]:
rhos.T == rhos

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [8]:
rhos

array([[ 0.29698109,  0.03198337, -0.05212048, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03198337,  0.29698109,  0.03198337, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05212048,  0.03198337,  0.29698109, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.13562352,
         0.04408787, -0.03850037],
       [ 0.        ,  0.        ,  0.        , ...,  0.04408787,
         0.13562352,  0.04408787],
       [ 0.        ,  0.        ,  0.        , ..., -0.03850037,
         0.04408787,  0.13562352]])

In [9]:
rhos.shape

(2516, 2516)

In [10]:
pickle.dump(rhos, open("tmp/rhos.pkl", "wb"))